In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import scipy.optimize as sc
from scipy.optimize import LinearConstraint
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def change(close, mean):
    last = close[-1]
    chg = mean/last - 1
    return chg

### Optimization

In [3]:
def get_data(stock_list, start, end):
    stock_list.sort()
    data = pdr.get_data_yahoo(stock_list, start, end)
    return data['Close']
def stock_performance(close):
    returns = close.pct_change()
    mean_returns = returns.mean()
    cov = returns.cov()
    return mean_returns, cov

def portfolio_performance(W, mean_returns, cov):
    W = np.asarray(W)
    portfolio_returns = (np.dot(W, mean_returns) * 252)
    portfolio_risk = np.sqrt(W.T @ cov @ W) * np.sqrt(252)
    return portfolio_returns, portfolio_risk  

############################
def negative_sharpe_ratio(W, mean_returns, cov, risk_free_rate):
    portfolio_return, portfolio_risk = portfolio_performance(W, mean_returns, cov)
    neg_sharpe_ratio = -(portfolio_return - risk_free_rate)/portfolio_risk
    return neg_sharpe_ratio

def optimize_portfolio(mean_returns, cov, upper_bound, risk_free_rate, ESG, esg_threshold):
    """
    returns
    -------
    sharpe_ratio, optimal_weights
    """
    #assign random weights
    np.random.seed(1)
    W = np.random.random(len(mean_returns))
    W = [weight/ np.sum(W) for weight in W]
    
    #add bounds
    bound = (0,upper_bound)
    bounds = tuple(bound for w in range(len(W)))
    
    #constraint
    def constraint(W):
        return np.sum(W) - 1
    
    def esg_constraint(W, ESG, esg_threshold):
        return np.sum(np.array(ESG)*np.array(W)) - esg_threshold

    constraint_set = [{'type': 'eq', 'fun': constraint}, {'type': 'ineq', 'fun': esg_constraint, 'args': [ESG, esg_threshold]}]
    #minimize negative SharpeRatio
    result = sc.minimize(negative_sharpe_ratio,
                        W,
                        args=(mean_returns, cov, risk_free_rate),
                        method='SLSQP',
                        bounds= bounds,
                        constraints=constraint_set)
    neg_sharpe_ratio, optimal_weights = result['fun'], result['x'].round(4)             
    return -neg_sharpe_ratio, optimal_weights   


######################
def minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate, ESG, esg_threshold):
    """
    returns
    -------
    sharpe_ratio, optimal_weights"""
     #assign random weights
    np.random.seed(1)
    W = np.random.random(len(mean_returns))
    W = [weight/ np.sum(W) for weight in W]

    #add bounds
    bound = (0,upper_bound)
    bounds = tuple(bound for w in range(len(W)))

    #constraints 
    def constraint(W):
        return np.sum(W) - 1
    
    def esg_constraint(W, ESG, esg_threshold):
        return np.sum(np.array(ESG)*np.array(W)) - esg_threshold
    
    constraint_set = [{'type': 'eq', 'fun': constraint}, {'type': 'ineq', 'fun': esg_constraint, 'args': [ESG, esg_threshold]}]
    
    def portfolio_variance(W,cov):
        return (np.sqrt(W.T @ cov @ W) * np.sqrt(252))

    result = sc.minimize(portfolio_variance,
                        W,
                        args = (cov),
                        bounds = bounds,
                        constraints = constraint_set,
                        method = 'SLSQP')

    sharpe_ratio, optimal_weights = result['fun'], result['x'].round(4)
    return -sharpe_ratio, optimal_weights


### Efficient market frontier

In [4]:
def simulate_portfolios(stock_list, start, end, simulations = 1000):
    
    close = get_data(stock_list, start, end)
    mean_returns, cov = stock_performance(close)
    
    returns = []
    risk = []
    weights = []
    for p in range(simulations):
        W = np.random.random(len(stock_list))
        W = W / np.sum(W)
        weights.append(W)
        
        portfolio_returns, portfolio_risk = portfolio_performance(W, mean_returns, cov)
        returns.append(portfolio_returns)
        risk.append(portfolio_risk)

    portfolios_weights = pd.DataFrame(weights, columns= stock_list)
    
    portfolios = pd.concat([pd.DataFrame({'Returns': returns, 'Risk': risk,}), portfolios_weights], axis = 1)
    return portfolios

def plot_EF(data,
            mean_returns,
            cov,
            stock_list,
            upper_bound,
            risk_free_rate,
            start,
            end,
            simulations = 10000):
            
    portfolios = simulate_portfolios(stock_list, start, end, simulations)
    
    #portfolios    
    fig = px.scatter(portfolios, portfolios['Risk'], portfolios['Returns'], hover_data=stock_list)
    
    #minimum varaince portfolio
    _, optimal_weights = minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate)
    minimum_risk_portfolio_return, minimum_risk_portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)
    #plot
    fig.add_trace(go.Scatter(x = [minimum_risk_portfolio_risk], y = [minimum_risk_portfolio_return],
                            mode = 'markers',
                            marker_symbol = 'star',
                            marker_size = 10,
                            marker = {'color' : 'red'}))

    #maximum sharpe ratio portfolio
    _, optimal_weights = optimize_portfolio(mean_returns, cov,upper_bound, risk_free_rate)
    optimum_portfolio_returns, optimum_portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)
    #plot
    fig.add_trace(go.Scatter(x=[optimum_portfolio_risk], y=[optimum_portfolio_returns], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 10,
                         marker = {'color' : 'red'}))
    fig.show() 

### Random walk simulation

In [5]:
def MC(close, steps, simulations, random_seed):
    daily_returns = np.log(1 + close.pct_change())
    mu = np.mean(daily_returns)
    var = np.var(daily_returns)
    sigma = np.std(daily_returns)
    drift = mu - 0.5*var

    returns = np.exp(drift + sigma * norm.ppf(np.random.rand(steps, simulations)))
    np.random.seed(random_seed)
    sims = np.zeros((steps, simulations))
    sims[0] = close.iloc[-1]

    for step in range(1,steps):
        sims[step] = sims[step-1] * returns[step]
    return sims


def monteCarlo(close, steps, simulations, random_seed):

    sims = MC(close=close, steps=steps, simulations=simulations, random_seed=random_seed)
    mcmean = np.mean(sims[-1])
    mcvar = np.var(sims[-1])
    return mcmean, mcvar 

In [6]:
def main(start, end, stock_list, risk_free_rate,  upper_bound, ESG, esg_threshold, simulations,  monte_carlo = False, steps = 90, random_seed = 1):
    """
    Paramters:
    ----------
    upper_bound: specifies upper bound for portfolio weights
    monte_carlo: whether to use monte carlo simulation to model stock price
    steps: number of days simualted in when using monte carlo
    """
    
    

    #bug fix
    yf.pdr_override()
    
    #get stock qoutes
    close = get_data(stock_list, start, end)

    mean_returns, cov = stock_performance(close)
    if monte_carlo: 

        mean_returns = []
        for i in range(close.shape[1]):
            c = close.iloc[:,i]
            mcmean, mcvar = monteCarlo(close=c, steps=steps,simulations=simulations,random_seed=random_seed)
            mean_returns.append(change(close=c, mean=mcmean))
        mean_returns = np.array(mean_returns)   
    #minimum variance porfolio    
    SR, optimal_weights = minimum_risk_portfolio(mean_returns, cov, upper_bound, risk_free_rate, ESG, esg_threshold)
    portfolio_returns, portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)

    print(f'Expected return: {portfolio_returns.round(3)}, Risk: {portfolio_risk.round(3)} with Sharpe Ratio:{SR.round(3)}\noptimal weights:')
    print(f'{close.columns.to_list()}\n{optimal_weights}')
    print('\n--------------\n')

    #maximum Sharpe Ratio portfolio
    SR, optimal_weights = optimize_portfolio(mean_returns, cov, upper_bound, risk_free_rate, ESG, esg_threshold)
    portfolio_returns, portfolio_risk = portfolio_performance(optimal_weights, mean_returns, cov)

    print(f'Expected return: {portfolio_returns.round(3)}, Risk: {portfolio_risk.round(3)} with Sharpe Ratio:{SR.round(3)}\noptimal weights:')
    print(f'{close.columns.to_list()}\n{optimal_weights}')
    
    df = pd.DataFrame({"ticker":close.columns.to_list(), "weight": optimal_weights})
    best_weights = df.loc[df['weight']>0,:].reset_index(drop=True)
    
    return best_weights

In [7]:
pooled = pd.read_excel("C:\\Users\\k_abo\\Downloads\\FinalPooled.xlsx", sheet_name = "pooled").sort_values(by='ticker').reset_index(drop=True)
pooled.dropna(subset="ESG", inplace = True)
stock_list = pooled['ticker'].to_list()
ESGs = pooled["ESG"]

In [8]:
pooled

,ticker,Name,GICS Sector,ESG,F-Score
0,A,Agilent Technologies,Health Care,42.0,5.0
1,ABC,AmerisourceBergen,Health Care,48.0,5.0
2,ABT,Abbott Laboratories,Health Care,52.0,6.0
3,AEP,"American Electric Power Co., Inc.",Utilities,51.0,5.0
4,ALB,Albemarle,Materials,49.0,6.0
...,...,...,...,...,...
114,WELL,Welltower,Real Estate,39.0,NaN
115,WFC,Wells Fargo,Financials,43.0,NaN
116,WM,Waste Management,Utilities,47.0,8.0
117,WMT,Wal-Mart Stores Inc,Consumer Staples,41.0,7.0


In [9]:
#time period
end = dt.datetime.now()
start = end - dt.timedelta(720)
simulations = 10000
upper_bound = 0.2
steps = 90 
random_seed = 1
risk_free_rate = 0.055
esg_threshold = 50

portfolio_weights = main(
    start=start,end=end,
    stock_list=stock_list, risk_free_rate=risk_free_rate,
    ESG=ESGs, esg_threshold=esg_threshold,
    simulations=simulations, monte_carlo=False,
    upper_bound=upper_bound, steps=steps,
    random_seed=random_seed
    )

portfolio_weights

[**********************97%********************** ]  113 of 116 completed

In [ ]:
portfolio = pooled.merge(portfolio_weights, on='ticker', how='right')
portfolio.to_csv("TSO.csv")

In [ ]:
np.sum(portfolio['weight']*portfolio['ESG'])

50.0001

In [ ]:
pooled = pd.read_excel("C:\\Users\\k_abo\\Downloads\\FinalPooled.xlsx", sheet_name = "TAA").sort_values(by='ticker').reset_index(drop=True)
pooled.dropna(subset="ESG", inplace = True)
stock_list = pooled['ticker'].to_list()
ESGs = pooled["ESG"]
#time period
end = dt.datetime.now()
start = end - dt.timedelta(720)
simulations = 10000
upper_bound = 0.2
steps = 90 
random_seed = 1
risk_free_rate = 0.055
esg_threshold = 50

portfolio_weights = main(
    start=start,end=end,
    stock_list=stock_list, risk_free_rate=risk_free_rate,
    ESG=ESGs, esg_threshold=esg_threshold,
    simulations=simulations, monte_carlo=False,
    upper_bound=upper_bound, steps=steps,
    random_seed=random_seed
    )

portfolio_weights

[*********************100%***********************]  58 of 58 completed
Expected return: 0.062, Risk: 0.109 with Sharpe Ratio:-0.109
optimal weights:
['A', 'ABC', 'ABT', 'AEP', 'AMGN', 'AMZN', 'AQN.TO', 'AWK', 'BAX', 'BBY', 'BMY', 'BSX', 'CAH', 'CMCSA', 'CNP', 'CPX.TO', 'CVS', 'DHR', 'DIS', 'DTE', 'ED', 'EIX', 'EMA.TO', 'ES', 'EW', 'EXC', 'FTS.TO', 'H.TO', 'HD', 'INE.TO', 'JNJ', 'JWN', 'LOW', 'M', 'MFI.TO', 'MRK', 'MTD', 'NEE', 'NI', 'NKE', 'NPI.TO', 'OMC', 'PEG', 'PFE', 'PVH', 'RNW.TO', 'RSG', 'SAP.TO', 'SO', 'SRE', 'TA.TO', 'TGT', 'TSCO', 'UA', 'WEC', 'WM', 'WMT', 'XEL']
[0.     0.     0.     0.     0.0433 0.     0.     0.     0.0701 0.
 0.     0.     0.     0.     0.     0.0069 0.     0.     0.0013 0.
 0.     0.     0.     0.     0.     0.     0.1086 0.2    0.     0.0292
 0.2    0.     0.     0.0033 0.     0.1609 0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.0627 0.     0.0319
 0.0676 0.     0.     0.     0.     0.0143 0.     0.    ]

--------------

Exp

,ticker,weight
0,ABC,0.2000
1,EXC,0.1178
2,H.TO,0.2000
3,JNJ,0.0721
4,M,0.0101
5,MRK,0.2000
6,MTD,0.0246
7,SRE,0.1753


In [ ]:
portfolio = pooled.merge(portfolio_weights, on='ticker', how='right')
portfolio.to_csv("TTSO.csv")
np.sum(portfolio['weight']*portfolio['ESG'])

49.995400000000004